## DSCI-511 - Final Project - Group 1

#### Financial Market Data Store

Data stored in Google Drive: [Access Link](https://drive.google.com/drive/folders/1hgWRHwlC9thoPKd7-dRqEHTPGYv3LtUk?usp=sharing)

##### Software Pre-requisites:
```
pip install pandas
pip install finnhub-python
pip install google-api-python-client
pip install gspread
pip install gspread-dataframe
pip install google-auth
pip install google-auth-oauthlib
```

##### Running instructions 
- Press `Run All` Button

In [ ]:
import time
import finnhub
import gspread
import gspread_dataframe as gd
import pandas as pd
import base64
from datetime import date, timedelta
from google.oauth2.service_account import Credentials
from googleapiclient import discovery
from googleapiclient.errors import HttpError

# dates for request parameters
start_time = time.time()
date_today = date.today()
today_date = date_today.strftime('%Y-%m-%d')
current_year = str(date_today.year)

from_time_unix = int(time.mktime((date_today - timedelta(weeks = 52)).timetuple()))
to_time_unix = int(time.mktime(date_today.timetuple()))

In [ ]:
# Finnhub clients setup - https://finnhub.io/docs/api/
def generate_finnhub_clients():
    finnhub_keys = ['Y2Q0b3FpYWFkM2k5OGpodTJwZ2djZDRvcWlhYWQzaTk4amh1MnBoMA==','Y2UzbWQyYWFkM2kxaDJuN24xODBjZTNtZDJhYWQzaTFoMm43bjE4Zw==','Y2R0ZG1yMmFkM2k0MXY3aG9nM2djZHRkbXIyYWQzaTQxdjdob2c0MA==',
        'Y2U1NmViaWFkM2lmZHZ0aHQzcjBjZTU2ZWJpYWQzaWZkdnRodDNyZw==','Y2U0dTFiMmFkM2llMTg4dGY4bmdjZTR1MWIyYWQzaWUxODh0ZjhvMA==','Y2UzbWU2MmFkM2kxaDJuN24xc2djZTNtZTYyYWQzaTFoMm43bjF0MA==',
        'Y2UzbWtiYWFkM2kxaDJuN240ZGdjZTNta2JhYWQzaTFoMm43bjRlMA==','Y2U1NjlxMmFkM2lmZHZ0aHQwZTBjZTU2OXEyYWQzaWZkdnRodDBlZw==','Y2UzbXYxYWFkM2kxaDJuN244dWdjZTNtdjFhYWQzaTFoMm43bjh2MA==',
        'Y2U0dHE5YWFkM2llMTg4dGY0YzBjZTR0cTlhYWQzaWUxODh0ZjRjZw==','Y2U0dTQyaWFkM2llMTg4dGZhYWdjZTR1NDJpYWQzaWUxODh0ZmFiMA==','Y2U1NmQ2cWFkM2lmZHZ0aHQzM2djZTU2ZDZxYWQzaWZkdnRodDM0MA==']

    finnhub_client_list = []

    for key in finnhub_keys:
        finnhub_client_list.append(finnhub.Client(api_key=base64.b64decode(key).decode()))

    return finnhub_client_list

finnhub_clients = generate_finnhub_clients()
client_num = 0

def get_finhub_client():
    """ returns a finnhub client to perform requests to gather financial data """

    global client_num
    if client_num >= len(finnhub_clients) - 1:
        client_num = 0
    else:
        client_num+=1

    return finnhub_clients[client_num]


In [ ]:
# google drive connection
def get_sa_info():
    sa_info = {
        "type": "service_account",
        "project_id": base64.b64decode('ZHNjaS01MTEtcHJvamVjdA==').decode(),
        "private_key_id": base64.b64decode("OGU5OWQyY2I4YzAwNmMyM2ZjYTU4YTNiOTE4MzA3YmM4NmQ3MDg1Yg==").decode(),
        "private_key": base64.b64decode('LS0tLS1CRUdJTiBQUklWQVRFIEtFWS0tLS0tCk1JSUV2UUlCQURBTkJna3Foa2lHOXcwQkFRRUZBQVNDQktjd2dnU2pBZ0VBQW9JQkFRRFVOOGpZUGoxUzdiTmkKczlQaFlxZFZld3g5RjJNU2tMY21WT2QzcCtJUGhNcGUzeUFPUjE0ODMrMkZFeXQwbVd3c0M5YWJJRUdydVJteApOem1PVE9Ca0NRR0dlRVJKTnVRL0poWEpyZlJBUWE5cVVKNWtFVkpnUjRkNnRKMG40YlJVaGc2T3c3VFhONFR6CmNwQjg5K1lzd1AzWEVQdlh4V0EyMGRpWVp2TEdwRktmenFpR2kzM3NZejNLc2J3eWNmYmdIZEdyeDZQajB0OHUKaEg3amJaU0ZOd0FBZllSanFhdWJSV1VFaFA5bzlIUno3TFd0N2orTk1JRXAvTmhMUXVCczBIVFVockxYT3oyawpIaEptZGp4SVZrc0VnWW9JRXJpb3hCc1pyOWFxMjRXeU1tWTV3cVh1QzBNejVZTHFEVExGbzVJc0Z0Wi92OC9LCnQ0STlxR2s1QWdNQkFBRUNnZ0VBRmxiaHBCaDMxblVpUEl6UjRZM0QycVRrMkNnd1FZSGlzeElkK1prVHhzUVkKNmlxOUVra04rNitBMlNZcG5XWm5IaTNkZzBxVHp5Z01NeWV0UGQ4TUxLRDJ2RTZJWUZmN0VwdGM1aDdvaHM1dAorNWczOXRzTW03T21reWYvSVNCMUhnUUI5VTBCdE9RVWNVSWVscUVoT3VzbjdYaWl0NENBcVNuOE9ITE1oMVM2Ck1VSXRHLzN6Y0QxZm1GL01IZ0F1SlAvV3RSOVVqTUNxVWtFY2VkdXN0RWZHNWVER3BoTmF5alM2ZExqc2JSTmkKRG9yY1ptT0s0eXY3d3hTQ3V0UFV1N0lHSlZ5V1V3ajJrSGZ2dEZEUDIvTHp1OG8xUWg5RG1YS2ZEcXhwWjJ6cwo5WE1GOVROcmFkK0lvclRIUmRnODdyQlNjK1F4M0x5Vmo1YWpBVExLeHdLQmdRRHFCZlk2dnlWajV5a3ZrQVMwClNRemw0Y3diSEFBbEN4WXFuQU11V3ZxenVSdklEbnl1d0dNN0NPbnczNVY3ajlOcjFCem40djhTUXdCNy93SloKaDZlYUd3MjIrMEwwZHMzejRVQVREbHZKU3BtaURPUXBxNlhZcU5Qd3dsYnRMSHZ4SWpoeGc3ZjFYeEV6NXJJWApyeldkazdtLzlDdDA1SCtZT2U2Y3R0dHhHd0tCZ1FEb0paeEFUYWIvMjh2N1RSOVhjT2hIczcxT05DdlkwVUlVCmhHemRxVGNvT0EycTU4MzNDK05Na0Faai9ldmF1dEFiOURkZnFQdWNNTTN6L3p5bkprUjlaZ3lSTG5Ga1FZcm8KcFlvb2t1ZjFKOFpPdE1MY0Qwdkttd0ROUlRDL0RHYy9VdzFuWHpDQ3RRMGdhdm9RUCtCQmR0QWVhdHRSQyt5Ygp4bFlUWGUySU93S0JnQ3BrcWhZNlNpdXUxR2NwVDdEWkN0MFo2ajkrOHdmQzJtRWRvZlhqeFVhSllkNzd3TnBuClA1S1NLczdYc2R5UVVWeFhNR1ZSV2FPVlA2RTJzTlpISExWdDhTNFVsaXh5MlJ3MDBjYWg4ejZqcFBFV0pMN24KdVVObVVmNXJOaFFBMnZMem9BWHdOb0xjbGVFeENWSUJDMVZnVFdYWWRZYlhYY1hWdU1QcTNCWEJBb0dBVDZBdQo0d04vaDMwTmowblp1VFZhQnRZSTFaMGhlUUgxcDNId29Xc2NKeWhxejdjb3QxY1ZoTHBWNXRBK3FNeXcreFdlCk5LcVNldlZtZncySjhVODM2TWNyQWxVcXdaZVZuTXpEQWIyNTRDWGZhRG1xS0s3NmNTa1BjbGtNS2t1SDBubWIKUEZ5ekYvSWY2WUduK2hEZXk3V0VGeHJSMVEvQS9ZRmZIczkxM09jQ2dZRUFoSnpyejlJM0ZTTHJxekNWKzhYVApLa00xNTg2dTZUMGdBM3hBME5ISVg5MkNHenRRT2lVSDd4Wjl2Q0pJTnNDVUs5S3JsVGlPWG1iMzJ5RU0vaTJlCkpqb0VsOWZYNjFUZVFrSzFpNmtBemJ0R2R0ZVlpaXRRMk5CQ3JRWVV1b2ZTRk1yeDY0OHEzaDRaZG5uUXk5VVAKa3p3OWk3cHBiMHJNeTNYS0RlbUZRZEE9Ci0tLS0tRU5EIFBSSVZBVEUgS0VZLS0tLS0K').decode(),
        "client_email": base64.b64decode('ZHNjaS01MTFAZHNjaS01MTEtcHJvamVjdC5pYW0uZ3NlcnZpY2VhY2NvdW50LmNvbQ==').decode(),
        "client_id": base64.b64decode('MTA4NTg5NzY5OTgzMDAzMTkwNTE2').decode(),
        "auth_uri": "https://accounts.google.com/o/oauth2/auth",
        "token_uri": "https://oauth2.googleapis.com/token",
        "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
        "client_x509_cert_url": base64.b64decode("aHR0cHM6Ly93d3cuZ29vZ2xlYXBpcy5jb20vcm9ib3QvdjEvbWV0YWRhdGEveDUwOS9kc2NpLTUxMSU0MGRzY2ktNTExLXByb2plY3QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20=").decode()
    }

    return sa_info

sa_creds = Credentials.from_service_account_info(get_sa_info(), scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])
gspread_client = gspread.authorize(sa_creds)
drive_service = discovery.build('drive', 'v3', credentials=sa_creds)

# google drive file IDs
basic_financials_file_id = base64.b64decode("MTh4bWY2NktFV25aQ3NzbmRXVUsxTHpvVG0tUUt2OG1VUXVueWczS2JnUEE=").decode()
company_profile_file_id = base64.b64decode("MUhtT1BNWm85anpPRm55Vjd5ZlRCSXI4RjJyMGRxcU5XZEViaG9qc3Ezcmc=").decode()
trends_file_id = base64.b64decode("MTQwNmN6V0FFWmpKYjcxdEp6bDBoeVF3SGIycldNS0JfZk1BSGJyVm9oV0U=").decode()


In [ ]:
def create_file(folder_id, ticker_symbol):
    file_id = None
    try:
        file_create_metadata = {
            'name': ticker_symbol,
            'parents': [folder_id],
            'mimeType': 'application/vnd.google-apps.spreadsheet',
        }
        file_id = drive_service.files().create(body=file_create_metadata).execute().get('id')

    except HttpError as error:
        print(f"Error while creating file for {ticker_symbol} in folder {folder_id}, error: {error}")
        time.sleep(5)
    
    return file_id

In [ ]:
def find_file(folder_id, ticker_symbol):
    file_id = None
    try:
        query_params = {"q": "'" + folder_id + "' in parents and mimeType != 'application/vnd.google-apps.folder'"}
        files = drive_service.files().list(**query_params).execute().get('files')

        for file in files:
            if file.get('mimeType') == 'application/vnd.google-apps.spreadsheet' and file.get('name') == ticker_symbol:
                file_id = file.get('id')
                break
                
    except HttpError as error:
        print(f"Error while searching folder {folder_id} for {ticker_symbol} file, error: {error}")
        time.sleep(5)
    
    return file_id

In [ ]:
def find_or_create_file(folder_id, ticker_symbol):
    file_id = find_file(folder_id, ticker_symbol)

    # create new file when no file is found
    if file_id == None:
        file_id = create_file(folder_id, ticker_symbol)

    return file_id

In [ ]:
def retrieve_existing_data_frame(file_id):

    existing_google_sheet = gspread_client.open_by_key(file_id).sheet1

    if existing_google_sheet.frozen_row_count == 0:
        existing_google_sheet.freeze(rows=1)

    if existing_google_sheet.row_count > 0:
        existing_data_frame = gd.get_as_dataframe(existing_google_sheet)
    else:
        existing_data_frame = pd.DataFrame()


    return existing_google_sheet, existing_data_frame

In [ ]:
def add_data_frame_to_empty_sheet(google_sheet, data_fame):
    google_sheet.update([data_fame.columns.values.tolist()] + data_fame.values.tolist())
    google_sheet.freeze(rows=1)


In [ ]:
def retrieve_company_peers(ticker_symbol):
    company_peers_response = get_finhub_client().company_peers(ticker_symbol)
    company_peers = None

    if company_peers_response and company_peers_response != None:
        company_peer_list = list(company_peers_response)

        if company_peer_list and len(company_peer_list) > 0:
            company_peers = " - ".join(company_peer_list)
    
    return company_peers

In [ ]:
company_profile_columns = ['symbol','name','country','finnhubIndustry','marketCapitalization','ipo','exchange','mic_code','security_type',
        'shareOutstanding','currency','company_peers','logo','weburl','phone','figi_identifier','update_date']

def populate_company_profile(ticker_symbol, stock_info):
    """ populates the general information for the given company """

    company_profile = get_finhub_client().company_profile2(symbol=ticker_symbol)

    profile_df = pd.json_normalize(company_profile)

    if not profile_df.empty:

        if 'ticker' in company_profile and len(company_profile['ticker']) > 0:
            company_peers = retrieve_company_peers(ticker_symbol)

            if company_peers and company_peers != None:
                profile_df['company_peers'] = company_peers

        if 'mic' in stock_info and len(stock_info['mic']) > 0:
            profile_df['mic_code'] = stock_info['mic']

        if 'figi' in stock_info and len(stock_info['figi']) > 0:
            profile_df['figi_identifier'] = stock_info['figi']

        if 'type' in stock_info and len(stock_info['type']) > 0:
            profile_df['security_type'] = stock_info['type']

        profile_df['update_date'] = today_date
        profile_df.rename(columns={"ticker": "symbol"}, inplace=True)

        try:
            google_sheet, existing_df = retrieve_existing_data_frame(company_profile_file_id)

            if not existing_df.empty:
                updated_df = pd.concat([existing_df, profile_df], axis=0).sort_values(by=['symbol', 'update_date']).drop_duplicates(subset=['symbol'], keep='last')[company_profile_columns]
                gd.set_with_dataframe(google_sheet, updated_df.dropna(how='all', axis=1))

            else:
                add_data_frame_to_empty_sheet(google_sheet, profile_df.sort_values(by=['symbol', 'update_date'])[company_profile_columns].dropna(how='all', axis=1))

        except Exception as error:
            print(f"Error while generating company profile for {ticker_symbol}, error: {error}")
            time.sleep(10)

In [ ]:
def retrieve_candlestick_data_frame(ticker_symbol):
    
    candle_response = get_finhub_client().stock_candles(symbol=ticker_symbol, resolution='D', _from=from_time_unix, to=to_time_unix)

    candles = pd.json_normalize(candle_response)[['t', 'o', 'c', 'h', 'l', 'v']]
    
    candles_df = pd.DataFrame(columns=['date','unix_time', 'open', 'close', 'high', 'low', 'volume'])
    
    candles_df['unix_time'] = candles['t'][0]
    candles_df['open'] = candles['o'][0]
    candles_df['close'] = candles['c'][0]
    candles_df['high'] = candles['h'][0]
    candles_df['low'] = candles['l'][0]
    candles_df['volume'] = candles['v'][0]

    candles_df.sort_values(by=['unix_time'], ascending=False, inplace=True)
    candles_df.drop_duplicates(subset=['unix_time'], keep='last', inplace=True)

    candles_df['date'] = pd.to_datetime(candles_df['unix_time'],unit='s').astype(str)    

    return candles_df

In [ ]:
def populate_candlestick_data(ticker_symbol):
    """ populates the daily candlestick data for the given stock """

    candlestick_df = retrieve_candlestick_data_frame(ticker_symbol)

    if not(candlestick_df.empty):
        candlestick_df.insert(0,'symbol', ticker_symbol)

        try:
            google_sheet, existing_df = retrieve_existing_data_frame("1zvSOtP7x-g7RSsA0yZ3Uj0ZzUy6zHtdBQauZ9LLC4X4")

            if not existing_df.empty:
                updated_df = pd.concat([existing_df, candlestick_df], axis=0).drop_duplicates(subset=['symbol', 'unix_time'], keep='last').sort_values(by=['symbol', 'unix_time'], ascending=[True, False])
                gd.set_with_dataframe(google_sheet, updated_df.dropna(how='all', axis=1))
                
            else:
                add_data_frame_to_empty_sheet(google_sheet, candlestick_df.sort_values(by=['symbol', 'unix_time'], ascending=[True, False]).dropna(how='all', axis=1))

        except Exception as error:
            print(f"Error while updating candlestick data for {ticker_symbol}, error: {error}")
            time.sleep(10)


In [ ]:
fin_report_columns = ['symbol','year','concept','label','value','form','cik']
financial_report_list = ['bs' , 'ic', 'cf']

def populate_financials_reported(ticker_symbol):
    """
    populates data into financials reported spreadsheet, the file generated will match the stock symbol using
    the data provided from the financials as reported endpoint for the given company
    """

    try:
        financials = get_finhub_client().financials_reported(symbol=ticker_symbol)

        google_sheet, existing_df = retrieve_existing_data_frame(find_or_create_file("1jRpA7By-7In1bSX1JkAbdlaQHaOyXz07", ticker_symbol))
        is_remote_df_empty = existing_df.empty

        for report_key in financial_report_list:
        
            for fin_entry in financials['data']:

                fin_report_df = pd.json_normalize(fin_entry['report'], record_path=[report_key])

                if not fin_report_df.empty:
                    fin_report_df.insert(0,'symbol', ticker_symbol)
                    fin_report_df.insert(1,'year', fin_entry['year'])
                    fin_report_df['form'] = fin_entry['form']
                    fin_report_df['cik'] = fin_entry['cik']
                    fin_report_df[fin_report_columns]

                    if not existing_df.empty:
                        existing_df = pd.concat([existing_df, fin_report_df], axis=0).drop_duplicates().sort_values(by=['symbol','year','label'], ascending=[True, False, True])[fin_report_columns]

                    else:
                        existing_df = fin_report_df.sort_values(by=['symbol','year','label'], ascending=[True, False, True])[fin_report_columns]


        if not is_remote_df_empty:
            gd.set_with_dataframe(google_sheet, existing_df)
        else:
            add_data_frame_to_empty_sheet(google_sheet, existing_df.sort_values(by=['symbol','year','label'], ascending=[True, False, True])[fin_report_columns])
            
    except Exception as error:
        print(f"Error while generating financials as reported for {ticker_symbol} error: {error}")
        time.sleep(10)

    

In [ ]:
renamed_social_columns = {"mention": "mention_twitter", "positiveScore": "positiveScore_twitter","negativeScore": "negativeScore_twitter",
    "positiveMention": "positiveMention_twitter","negativeMention": "negativeMention_twitter","score": "score_twitter"}

def retrieve_social_sentiment_data_frame(ticker_symbol):

    social_response = get_finhub_client().stock_social_sentiment(ticker_symbol)

    twitter_social_df = pd.json_normalize(social_response, record_path='twitter')
    reddit_social_df = pd.json_normalize(social_response, record_path='reddit')

    try:
        if 'atTime' in reddit_social_df:
            social_df = twitter_social_df.merge(reddit_social_df, how='left', on=['atTime'], suffixes=('', '_reddit')).fillna(0).rename(columns=renamed_social_columns)

        else:
            social_df = twitter_social_df.fillna(0).rename(columns=renamed_social_columns)

    except Exception as error:
        print(f"Error while gathering social sentiment for {ticker_symbol} error: {error}")
        social_df = twitter_social_df.fillna(0).rename(columns=renamed_social_columns)

    return social_df

In [ ]:
def populate_social_sentiment(ticker_symbol):
    """ populates the social sentiment for stocks on Reddit and Twitter for the given stock """

    social_df = retrieve_social_sentiment_data_frame(ticker_symbol)
    if not(social_df.empty):
        
        social_df.insert(0,'symbol', ticker_symbol)

        try:
            google_sheet, existing_df = retrieve_existing_data_frame("1-SXSPy03c9lAvSTBdSIl2up0eN1oQacnQXenInUopxo")

            if not existing_df.empty:
                updated_df = pd.concat([existing_df, social_df], axis=0).drop_duplicates(subset=['symbol', 'atTime'], keep='last').sort_values(by=['symbol', 'atTime'], ascending=[True, False]).dropna(how='all', axis=1).fillna(0)
                gd.set_with_dataframe(google_sheet, updated_df)

            else:
                add_data_frame_to_empty_sheet(google_sheet, social_df.sort_values(by=['symbol', 'atTime'], ascending=[True, False]).dropna(how='all', axis=1))

        except Exception as error:
            print(f"Error while generating social sentiment data for {ticker_symbol}, error: {error}")
            time.sleep(10)


In [ ]:
insider_trans_cols =['symbol','share','change','transactionDate','transactionCode','transactionPrice','name','filingDate','id']

def populate_insider_transactions(ticker_symbol):
    """ populates insider transactions """

    insider_transactions_response = get_finhub_client().stock_insider_transactions(ticker_symbol)

    insider_trans_df = pd.json_normalize(insider_transactions_response, record_path='data')

    if not insider_trans_df.empty:
        try:
            google_sheet, existing_df = retrieve_existing_data_frame("1ivT0a63oHomkACLQIeq5tbFR2VET6dyRkQSsl6LWmoo")

            if not existing_df.empty:
                updated_df = pd.concat([existing_df, insider_trans_df], axis=0).drop_duplicates().sort_values(by=['symbol','transactionDate'], ascending=[True, False]).dropna(how='all', axis=1)[insider_trans_cols]
                gd.set_with_dataframe(google_sheet, updated_df)

            else:
                add_data_frame_to_empty_sheet(google_sheet, insider_trans_df.sort_values(by=['symbol','transactionDate'], ascending=[True, False]).dropna(how='all', axis=1)[insider_trans_cols])

        except Exception as error:
            print(f"Error while generating insider transactions data for {ticker_symbol}, error: {error}")
            time.sleep(10)

In [ ]:
earnings_cols =['symbol','period','actual','estimate','surprise','surprisePercent']

def populate_surprise_earnings(ticker_symbol):
    """ populates a company's surprise earnings """

    earnings_response = get_finhub_client().company_earnings(ticker_symbol)

    earnings_df = pd.json_normalize(earnings_response)

    if not earnings_df.empty:
        try:
            google_sheet, existing_df = retrieve_existing_data_frame("1h9ypbJSdq1E-y5B-7zAGtQI8pZovupYuEpjECLK95y0")

            if not existing_df.empty:
                updated_df = pd.concat([existing_df, earnings_df], axis=0).drop_duplicates(subset=['symbol', 'period']).sort_values(by=['symbol','period'], ascending=[True, False]).dropna(how='all', axis=1)[earnings_cols]
                gd.set_with_dataframe(google_sheet, updated_df)

            else:
                add_data_frame_to_empty_sheet(google_sheet, earnings_df.sort_values(by=['symbol','period'], ascending=[True, False]).dropna(how='all', axis=1)[earnings_cols])

        except Exception as error:
            print(f"Error while generating company surprise earnings data for {ticker_symbol}, error: {error}")
            time.sleep(10)

In [ ]:
def populate_basic_financials(ticker_symbol):
    """ populates the basic financials of a company """

    basic_financials_response = get_finhub_client().company_basic_financials(ticker_symbol, 'all')

    if 'metric' in basic_financials_response and 'symbol' in basic_financials_response:

        basic_fin_df = pd.DataFrame([basic_financials_response['metric']])
        basic_fin_df.insert(0,'symbol', basic_financials_response['symbol'])
        basic_fin_df['update_date'] = today_date

        try:
            google_sheet, existing_df = retrieve_existing_data_frame(basic_financials_file_id)

            if not existing_df.empty:
                updated_df = pd.concat([existing_df, basic_fin_df], axis=0).sort_values(by=['symbol', 'update_date'], ascending=[True, False]).drop_duplicates(subset=['symbol'], keep='last').dropna(how='all', axis=1)
                gd.set_with_dataframe(google_sheet, updated_df)

            else:
                add_data_frame_to_empty_sheet(google_sheet, basic_fin_df.dropna(how='all', axis=1))

        except Exception as error:
            print(f"Error while updating basic financial data for {ticker_symbol}, error: {error}")
            time.sleep(10)


In [ ]:
def populate_insider_sentiment(ticker_symbol):
    """ populates the insider sentiment data """

    insider_response = get_finhub_client().stock_insider_sentiment(ticker_symbol, "2010-01-01", current_year+"-12-31")

    insider_df = pd.json_normalize(insider_response, record_path='data')

    if not insider_df.empty:
        try:
            google_sheet, existing_df = retrieve_existing_data_frame("197PQPaYIJeCCYGHJreqA1duHcPfL5Ql5IhVj2JdSyuA")

            if not existing_df.empty:
                updated_df = pd.concat([existing_df, insider_df], axis=0).drop_duplicates().sort_values(by=['symbol','year','month'], ascending=[True,False,False]).dropna(how='all', axis=1)
                gd.set_with_dataframe(google_sheet, updated_df)

            else:
                add_data_frame_to_empty_sheet(google_sheet, insider_df.sort_values(by=['symbol','year','month'], ascending=[True,False,False]).dropna(how='all', axis=1))

        except Exception as error:
            print(f"Error while generating insider sentiment data for {ticker_symbol}, error: {error}")
            time.sleep(10)

In [ ]:
trends_columns = ['symbol', 'period', 'strongBuy', 'buy', 'hold', 'sell', 'strongSell']

def populate_recommended_trends(ticker_symbol):
    """ populates the latest analyst recommendation trends for a company """

    trends = get_finhub_client().recommendation_trends(symbol=ticker_symbol)
    trends_df = pd.json_normalize(trends)

    if not trends_df.empty:

        trends_df[trends_columns]

        try:
            google_sheet, existing_df = retrieve_existing_data_frame(trends_file_id)
            
            if not existing_df.empty:
                updated_df = pd.concat([existing_df, trends_df], axis=0).sort_values(by=['symbol', 'period'], ascending=[True, False]).drop_duplicates(subset=['symbol', 'period'])[trends_columns]
                gd.set_with_dataframe(google_sheet, updated_df.dropna(how='all', axis=1))

            else:
                add_data_frame_to_empty_sheet(google_sheet, trends_df.sort_values(by=['symbol', 'period'], ascending=[True, False]).dropna(how='all', axis=1))

        except Exception as error:
            print(f"Error while updating recommendation trends data for {ticker_symbol}, error: {error}")
            time.sleep(10)


In [ ]:
def populate_senate_lobbying(ticker_symbol):
    """ populates the reported lobbying activities in the Senate and the House """

    lobby_response = get_finhub_client().stock_lobbying(ticker_symbol, "2000-01-01", today_date)

    lobby_df = pd.json_normalize(lobby_response, record_path='data')

    if not lobby_df.empty:

        symbol_col = lobby_df.pop('symbol')
        year_col = lobby_df.pop('year')
        lobby_df.insert(0, 'symbol', symbol_col)
        lobby_df.insert(0, 'year', year_col)

        try:
            google_sheet, existing_df = retrieve_existing_data_frame("1rY_9WQ5F80BEzpoVRbIM739sXqrbEj84XJ7EllbvCF0")

            if not existing_df.empty:
                updated_df = pd.concat([existing_df, lobby_df], axis=0).sort_values(by=['symbol', 'year'], ascending=[True, False]).drop_duplicates().dropna(how='all', axis=1)
                gd.set_with_dataframe(google_sheet, updated_df)
                
            else:
                add_data_frame_to_empty_sheet(google_sheet, lobby_df.dropna(how='all', axis=1))

        except Exception as error:
            print(f"Error while generating senate lobbying data for {ticker_symbol}, error: {error}")
            time.sleep(10)

In [ ]:
def should_generate_data_for_ticker(stock_info, symbol_input, stock_tickers):
    
    is_stock_info_valid = 'symbol' in stock_info and len(stock_info['symbol']) > 0

    if is_stock_info_valid:

        if len(stock_tickers) > 0 and stock_info['symbol'] in stock_tickers:
            return True

        has_symbol_input = symbol_input and len(symbol_input) > 0

        if not(has_symbol_input):
            return True
        
        if has_symbol_input and symbol_input == stock_info['symbol']:
            return True
    
    return False

In [ ]:
def prompt_user_for_symbol_input():

    user_symbol_input = None
    
    print("-- Enter Stock Symbol In Prompt -- \nIf no input is provided data will be generated from a pre-defined set of stocks")

    try:
        user_symbol_input = input("Enter Stock Symbol or Nothing for Nasdaq Stocks:")
    
    except Exception as e:
        print("An error occured while waiting for input. Data will be generated multiple stocks")
    

    if user_symbol_input and len(user_symbol_input) > 0:
        print(f"Attempting to Generate Data Stock Symbol: {user_symbol_input}")
    else:
        print("No Stock Symbol Provided.")

    return user_symbol_input



In [ ]:
def should_generate_data_for_ticker2(stock_info, stock_tickers):
    return 'symbol' in stock_info and len(stock_info['symbol']) > 0 and stock_info['symbol'] in stock_tickers

In [51]:
# retrieves all stock ticker symbols matching the params
all_stock_info = get_finhub_client().stock_symbols(exchange="US", currency="USD", security_type="Common Stock", mic="XNAS")
# mic="XNAS" - Nasdaq, mic="XNYS" - New York Stock Exchange

stock_tickers = {}

symbol_input = prompt_user_for_symbol_input()

print("Please Wait... Generating Financial Data. Interrupt the Program to Exit")
processed_symbols = []

for stock_info in all_stock_info:
    try:
        if should_generate_data_for_ticker(stock_info, stock_tickers, stock_tickers):
            ticker_symbol = stock_info['symbol']

            populate_company_profile(ticker_symbol, stock_info)
            
            populate_candlestick_data(ticker_symbol)
            
            populate_financials_reported(ticker_symbol)
            
            populate_social_sentiment(ticker_symbol)

            populate_insider_transactions(ticker_symbol)

            populate_surprise_earnings(ticker_symbol)

            populate_basic_financials(ticker_symbol)

            populate_insider_sentiment(ticker_symbol)
            
            populate_recommended_trends(ticker_symbol)

            populate_senate_lobbying(ticker_symbol)

            processed_symbols.append(ticker_symbol)

            if len(processed_symbols) % 5 == 0:
                print(f"{len(processed_symbols)} Symbols Processed. Current Run Time: {(time.time() - start_time) / 60} minutes")

    except Exception as e:
        print(f"Error while generating data for {ticker_symbol}, error: {e}")
        time.sleep(10)
    except KeyboardInterrupt:
        print("Process interrupted")
        break

print(f"-- Exiting Program -- Total Execution Time: {(time.time() - start_time) / 60} minutes")
print(f"{len(processed_symbols)} Stock Symbols Processed. Symbol List: {processed_symbols}")
